In [2]:
#Libraries
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
from sklearn.datasets import load_iris


In [3]:
# For reproducibility
nnfs.init()

COMPONENT CLASSES (LAYERS, ACTIVATIONS, LOSS, OPTIMIZERS)

In [4]:
# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)


# ReLU activation
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


# Softmax classifier - combined Softmax activation and cross-entropy loss
# for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples

# Softmax activation
class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


# Common loss class
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


# Cross-entropy loss
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

Modify the Optimizer class for 3 optimizers.
The two classes below fulfill this requirement.

In [5]:
# Optimizer: Stochastic Gradient Descent (SGD)
# This class includes logic for:
#   a) Learning rate decay (in pre_update_params)
#   b) Momentum (in update_params)
class Optimizer_SGD:
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        # This is where learning rate decay is calculated
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        # If momentum is used, this block is executed
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        # Vanilla SGD update (without momentum)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
        
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


# Optimizer: Adaptive Gradient (Adagrad)
# This class implements:
#   c) Adaptive Gradient
class Optimizer_Adagrad:
    def __init__(self, learning_rate=1., decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Adagrad update rule
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

DATASET AND NETWORK INITIALIZATION

In [17]:
X, y = spiral_data(samples=100, classes=3)
dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# You can switch between the optimizers by uncommenting one of the lines below.

#optimizer = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.9)
optimizer = Optimizer_Adagrad(learning_rate=1.0, decay=1e-4)

Perform FP, BP, and weight updates in a loop (1000+ epochs)

In [18]:
for epoch in range(10001):

    # --- FORWARD PASS (FP) ---
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, y)

    # --- PERFORMANCE METRICS ---
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y_true = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)

    # =========================================================================
    # REQUIREMENT 3: Display the accuracy once every 100 epochs
    # The `if` block below fulfills this requirement.
    # =========================================================================
    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate:.4f}')

    # --- BACKPROPAGATION (BP) ---
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # --- UPDATE WEIGHTS ---
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

    accuracy = np.mean(predictions == y)

epoch: 0, acc: 0.350, loss: 1.099, lr: 1.0000
epoch: 100, acc: 0.500, loss: 0.930, lr: 0.9902
epoch: 200, acc: 0.577, loss: 0.863, lr: 0.9805
epoch: 300, acc: 0.567, loss: 0.813, lr: 0.9710
epoch: 400, acc: 0.577, loss: 0.779, lr: 0.9616
epoch: 500, acc: 0.680, loss: 0.714, lr: 0.9525
epoch: 600, acc: 0.637, loss: 0.687, lr: 0.9435
epoch: 700, acc: 0.660, loss: 0.661, lr: 0.9347
epoch: 800, acc: 0.723, loss: 0.616, lr: 0.9260
epoch: 900, acc: 0.727, loss: 0.596, lr: 0.9175
epoch: 1000, acc: 0.737, loss: 0.574, lr: 0.9092
epoch: 1100, acc: 0.647, loss: 0.709, lr: 0.9010
epoch: 1200, acc: 0.747, loss: 0.538, lr: 0.8929
epoch: 1300, acc: 0.753, loss: 0.523, lr: 0.8850
epoch: 1400, acc: 0.780, loss: 0.508, lr: 0.8773
epoch: 1500, acc: 0.787, loss: 0.493, lr: 0.8696
epoch: 1600, acc: 0.793, loss: 0.478, lr: 0.8621
epoch: 1700, acc: 0.797, loss: 0.467, lr: 0.8548
epoch: 1800, acc: 0.797, loss: 0.457, lr: 0.8475
epoch: 1900, acc: 0.813, loss: 0.442, lr: 0.8404
epoch: 2000, acc: 0.807, loss: 0